# Analyze Antibody Escape Variants

This notebook analyzes and plots viral variants in the genome after passaging under various conditions.

In [5]:
import os
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [15]:
def parse_genes_from_gff(gff):
    """
    Parse a gene positions from a gff file.
    """
    # Read in GFF file from path
    protein_coordinates = []
    with open(gff, 'r') as f:
        gff = f.readlines()
        for line in gff:
            if line.strip() == '' or line.startswith('#'):
                continue
            parts = line.strip().split('\t')
            if parts[2] == 'CDS':
                protein_coordinates.append(parts)
        
    # Extract the gene from the genome
    coordinates = []
    for protein in protein_coordinates:
        start, end = int(protein[3]), int(protein[4])
        for part in protein[8].split(';'):
            if 'gene=' in part:
                gene = part.split('=')[1]
                break
        coordinates.append({
            'Gene': gene,
            'Start': start,
            'Stop': end
        })
    return pd.DataFrame(coordinates)

Notebook parameters:

In [22]:
if 'snakemake' in globals():
    # Virus to analyze
    virus = snakemake.wildcards.virus
    # Input Runs
    variant_files = snakemake.input.variant_files
    # GFF Gene Annotations
    gff = snakemake.input.gff
    # Output plot
    all_variants = snakemake.output.all_variants
    # Sample names
    samples = snakemake.params.samples
    # Parameters
    min_pval = snakemake.params.min_pval
    min_depth = snakemake.params.min_depth
    min_freq = snakemake.params.min_freq
else:
    # Virus to analyze
    virus = "RSV-B"
    # Input Runs
    variant_files = [os.path.join(f"../../results/variants/{virus}/", f) for f in os.listdir(f"../../results/variants/{virus}/") if f.endswith('.tsv')]
    print(variant_files)
    # GFF Gene Annotations
    gff = f"../../results/reference/{virus}/{virus}.gff"
    # Output directory
    os.makedirs(f"../../results/summary/{virus}", exist_ok=True)
    all_variants = os.path.join("../../results/summary/{virus}", 'all_variants.html')
    # Sample names
    samples_df = pd.read_csv(os.path.join('../../configuration', 'samples.csv'))
    samples = [sample.Run for _, sample in samples_df.iterrows() if sample.Virus == virus]
    print(samples)
    # Parameters
    min_pval = 0.05
    min_depth = 200
    min_freq = 0.03

['../../results/variants/RSV-B/MxR2.tsv', '../../results/variants/RSV-B/Pali30.tsv', '../../results/variants/RSV-B/Stock.tsv', '../../results/variants/RSV-B/Pali15.tsv', '../../results/variants/RSV-B/MxR4.tsv', '../../results/variants/RSV-B/Control.tsv']
['Stock', 'Control', 'Pali30', 'Pali15', 'MxR4', 'MxR2']


Get the location of each gene to annotate on the plot:

In [23]:
gene_annotations = parse_genes_from_gff(gff)

Get the names of the conditions being analyzed:

In [24]:
assert "Stock" in samples, "Stock sample not found in samples.csv"
print(f"Analyzing the following conditions: {samples}")

Analyzing the following conditions: ['Stock', 'Control', 'Pali30', 'Pali15', 'MxR4', 'MxR2']


Import and process the variant calls into a single table:

In [28]:
assert len(variant_files) == len(samples), "Number of variant files does not match number of samples"

# Load variants and join into a single table
dfs = []
for file in variant_files:
    sample = os.path.basename(file).split('.')[0]
    assert sample in samples, f"Sample {sample} not in samples"
    df = pd.read_csv(file, sep='\t')
    # Parse the GFF_FEATURE field by splitting by ":" and taking the first element if x isn't NaN
    df['GFF_FEATURE'] = df['GFF_FEATURE'].apply(lambda x: x.split(":")[0] if not pd.isna(x) else x)
    df.rename(columns={'GFF_FEATURE': 'GENE'}, inplace=True)
    # Add a column for the sample name
    df['SAMPLE'] = sample    
    dfs.append(df)
variants_df = pd.concat(dfs)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 150846: invalid start byte

Filter out variants that aren't statistically significant:

In [ ]:
# Determine the mutation type (Synonymous, Nonsynonymous, Stop)
def determine_mutation_type(row):
    if pd.isna(row['REF_AA']) and pd.isna(row['ALT_AA']):
        return 'Synonymous'
    elif row['REF_AA'] == row['ALT_AA']:
        return 'Synonymous'
    elif row['REF_AA'] != row['ALT_AA'] and row['ALT_AA'] != '*':
        return 'Nonsynonymous'
    else:
        return 'Stop'
variants_df['MUTATION_TYPE'] = variants_df.apply(determine_mutation_type, axis=1)

In [ ]:
# Make new column called MUTATION_ANNOTATION
def mutation_annotation(row):
    if pd.isna(row['REF_AA']) and pd.isna(row['ALT_AA']):
        return f"nuc:{row['REF']}{int(row['POS'])}{row['ALT']}"
    else:
        return f"{row['GENE']}:{row['REF_AA']}{int(row['POS_AA'])}{row['ALT_AA']}"
variants_df['MUTATION_ANNOTATION'] = variants_df.apply(mutation_annotation, axis=1)

In [ ]:
# Add a new column for InDels/SNPs
def determine_mutation_class(row):
    if len(row['REF']) == len(row['ALT']):
        return False
    else:
        return True
    
variants_df['MUTATION_CLASS'] = variants_df.apply(determine_mutation_class, axis=1)


## Plot All Variants

Below, I'll plot all of the variants from each sample in the sample plot. I've applied some default filters, but the values can be changed using the menu below the plot. Click on a condition in the legend to only show that condition on the plot. Click and drag on the zoom bar to focus into specific regions of the genome.

In [ ]:
# Selection for zoom
zoom_selection = alt.selection_interval(encodings=["x"], mark=alt.BrushConfig(stroke='black', strokeWidth=2))

# Filter based on total depth
depth_slider = alt.param(
    value=min_depth,
    bind=alt.binding_range(
        name="Minimum Depth",
        min=1,
        max=variants_df['TOTAL_DP'].max(),
    ),
)

# Filter based on minimum p-value
pvalue_slider = alt.param(
    value=min_pval,
    bind=alt.binding_range(
        name="Minimum P-value",
        min=variants_df['PVAL'].min(),
        max=variants_df['PVAL'].max(),
    ),
)

# Filter based on minimum allele frequency
freq_slider = alt.param(
    value=min_freq,
    bind=alt.binding_range(
        name="Minimum Frequency",
        min=variants_df['ALT_FREQ'].min(),
        max=variants_df['ALT_FREQ'].max(),
    ),
)

# Filter based on InDels
indel_checkbox = alt.param(bind=alt.binding_checkbox(name='Show Insertions and Deletions: '), value=False)

# Filter based on mutation type
mutation_dropdown = alt.binding_select(options=[None] + [type for type in set(variants_df['MUTATION_TYPE'].to_list())], labels = ['All'] + [type for type in set(variants_df['MUTATION_TYPE'].to_list())], name='Mutation Type  ')
mutation_selection = alt.selection_point(fields=['MUTATION_TYPE'], bind=mutation_dropdown)


# Filter based on condition
sample_selection = alt.selection_point(fields=['SAMPLE'], bind='legend')
sample_opacity = alt.condition(
    sample_selection,
    alt.value(1),
    alt.value(0)
)

# Variant points
points = alt.Chart(variants_df).mark_circle(
    size = 50
).encode(
    x=alt.X('POS:Q', title='Position', axis=alt.Axis(labels=True, labelAngle=-45)).scale(domain=zoom_selection, domainMax=variants_df['POS'].max()+50, domainMin=variants_df['POS'].min()-50),
    y=alt.Y('ALT_FREQ:Q', title='Allele Frequency', axis=alt.Axis(labels=True, grid=False)),
    color=alt.Color('SAMPLE:N', title='Condition'),
    opacity=sample_opacity,
    tooltip=[
        alt.Tooltip('SAMPLE', title='Sample:'),
        alt.Tooltip('MUTATION_ANNOTATION', title='Mutation:'),
        alt.Tooltip('PVAL', title='P-Value:')
    ]
).add_params(
    depth_slider, pvalue_slider, freq_slider, sample_selection, mutation_selection, indel_checkbox
).transform_filter(
    alt.datum["TOTAL_DP"] >= depth_slider
).transform_filter(
    alt.datum["PVAL"] < pvalue_slider
).transform_filter(
    alt.datum["ALT_FREQ"] >= freq_slider
).transform_filter(
    alt.datum["MUTATION_CLASS"] == indel_checkbox
).transform_filter(
    mutation_selection
).properties(
    width=1200,
    height=200
)

# Zoom bar
zoom = (
    alt.Chart(gene_annotations)
    .mark_rect()
    .encode(
        x=alt.X('Start', title='Zoom Bar', scale=alt.Scale(domainMax=variants_df['POS'].max()+50, domainMin=variants_df['POS'].min()-50)),
        x2=alt.X2('Stop'),
        color=alt.Color(
            'Gene',
            type="nominal",
            scale=alt.Scale(scheme='set3'),
            legend=alt.Legend(orient="top"),
        ),
        tooltip=[
            alt.Tooltip('Gene', title='Gene:'),
        ]
    )
    .properties(width=1200, height=15)
    .add_params(
        zoom_selection
    )
)

# Combine charts vertically
chart = (zoom & points).resolve_scale(color="independent")
chart = chart.properties(
    title=f"{virus} Variant Allele Frequencies",
)
chart.display()

alt.VConcatChart(...)

In [ ]:
chart.save(all_variants)